# Comparison with other datasets

In [ ]:
import functools, session_info
import numpy as np
import pandas as pd
from plotnine import *
from pyhere import here
from scipy.stats import binom_test, fisher_exact, linregress

from warnings import filterwarnings
from matplotlib.cbook import mplDeprecation
filterwarnings('ignore', category=mplDeprecation)
filterwarnings('ignore', category=UserWarning, module='plotnine.*')
filterwarnings('ignore', category=DeprecationWarning, module='plotnine.*')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

## BrainSeq functions

In [ ]:
@functools.lru_cache()
def get_biomart():
    return pd.read_csv('../_h/biomart.csv', index_col=0)


@functools.lru_cache()
def get_deg(tissue):
    fname = f'../../../{tissue}/_m/genes/diffExpr_maleVfemale_full.txt'
    dft = pd.read_csv(fname, sep='\t', index_col=0)
    dft['Feature'] = dft.index
    dft['Direction'] = np.sign(dft['t'])
    dft['ensemblID'] = dft.gencodeID.str.replace('\\..*', '', regex=True)
    return dft[['Feature', 'ensemblID', 'Symbol', 'adj.P.Val', 'logFC', 't', 'Direction']]


@functools.lru_cache()
def get_deg_sig(tissue, fdr):
    dft = get_deg(tissue)
    return dft[(dft['adj.P.Val'] < fdr)]


@functools.lru_cache()
def merge_dataframes(tissue1, tissue2, fdr=1):
    return get_deg_sig(tissue1, fdr)\
    .merge(get_deg_sig(tissue2, fdr), on='Feature', 
           suffixes=[f'_{tissue1}', f'_{tissue2}'])


@functools.lru_cache()
def brainseq_shared_degs():
    fn = "../../summary_table/_m/differential_expression_analysis_4features_sex.txt.gz"
    df0 = pd.read_csv(fn, sep='\t')
    df = df0.loc[(df0["Type"] == "Gene") & (df0["adj.P.Val"] < 0.05),
                 ["ensemblID", "Tissue"]].copy()
    df["DE"] = 1
    return df.pivot(index="ensemblID", columns="Tissue", values="DE").fillna(0)

## Trabzuni comparison

In [ ]:
@functools.lru_cache()
def get_symbols(tissue, fdr):
    return get_deg_sig(tissue, fdr)[['Symbol']]


def tissue_annotation(tissue):
    return {'dlpfc': 'DLPFC', 'hippocampus': 'Hippocampus', 
            'caudate': 'Caudate', 'cmc_dlpfc': 'CMC DLPFC'}[tissue]

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """
    
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False, labelsize=18,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = mpl.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
config_trabzuni = {
    'genes': here('input/public_results/extract_geneLists/_m/trabzuni_sex_diff_genes_logFC.csv'),
    'annot': here('input/public_results/_m/trabzuni/trabzuni_annotation.txt'),
}
    
zz = dict()
for tissue in ['caudate', 'dlpfc', 'hippocampus']:
    ww = []; yy = []
    fdr = 0.05
    trabzuni = pd.read_csv(config_trabzuni['genes'], index_col=0)\
                 .drop(['average ALL'], axis=1)
    annot = pd.read_csv(config_trabzuni['annot'], sep='\t', index_col=0)
    for col in trabzuni.drop('Chromosome', axis=1).columns:
        trabzuni.loc[trabzuni[col] != 0, col] = 1
        tot = len(set(trabzuni[(trabzuni[col] != 0)].index))
        overlap = len(set(get_symbols(tissue, fdr).Symbol) & 
                      set(trabzuni[(trabzuni[col] != 0)].index))
        xx = overlap / tot 
        ww.append(xx)
        yy.append(annot.to_dict()['TISSUE'][col.split(' ')[0]])
        print("There is %d (%.1f%%) overlap between %s and %s %s!" % 
              (overlap, xx* 100, tissue_annotation(tissue), 
               annot.to_dict()['TISSUE'][col.split(' ')[0]], 
               col.split(" ")[1]))
    print("")
    zz[tissue_annotation(tissue)] = ww

In [ ]:
dfz = pd.DataFrame.from_dict(zz)
dfz['Tissue'] = trabzuni.drop('Chromosome', axis=1).columns
dfz = dfz.set_index('Tissue')
dfz

In [ ]:
dfz.to_csv('brainseq_overlap_trabzuni_fraction.txt', sep='\t')

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(8, 12))
plt.rcParams.update({'font.size': 18})
im, cbar = heatmap(dfz, dfz.index, dfz.columns, ax=ax,
                   cmap="YlGn", cbarlabel="Trabzuni et al. Comparison")
texts = annotate_heatmap(im, valfmt="{x:.2f}")

fig.tight_layout()
plt.savefig('brainseq_overlap_trabzuni_heatmap.png')
plt.savefig('brainseq_overlap_trabzuni_heatmap.pdf')
plt.savefig('brainseq_overlap_trabzuni_heatmap.svg')
plt.show()

In [ ]:
df = brainseq_shared_degs()
df.to_csv('brainseq_deg_across_tissues_comparison.csv')
df.head(2)

In [ ]:
tt0 = get_biomart()\
    .merge(df, left_on='ensembl_gene_id', right_index=True)\
    .merge(trabzuni, left_on='external_gene_name', right_index=True, how='right')\
    .set_index('external_gene_name')

tt1 = tt0[['Caudate', 'DLPFC', 'Hippocampus']].fillna(0)
tt2 = tt0[['CRBL (N=128)', 'FCTX (N=122)', 'HIPP (N=114)', 'HYPO (N=13)', 
           'MEDU (N=109)', 'OCTX (N=125)', 'PUTM (N=121)', 'SNIG (N=96)', 
           'TCTX (N=114)', 'THAL (N=107)', 'WHMT (N=120)']].fillna(0)
tt = pd.concat([tt1, tt2], axis=1)
tt.to_csv('brainseq_overlap_trabzuni_genes.csv')
tt.head(2)

## Mayne comparison

In [ ]:
mayne_file = here('input/public_results/extract_geneLists/_m/mayne_sex_diff_genes.csv')
mayne = pd.read_csv(mayne_file)
mayne.groupby('Version').size()

In [ ]:
zz = dict()
for tissue in ['caudate', 'dlpfc', 'hippocampus']:
    ww = []; yy = []
    fdr = 0.05
    mayne = pd.read_csv(mayne_file)
    mayne['Mayne'] = 1
    for ver in ['all', 'autosomes', 'noY']:
        mm = mayne.loc[(mayne['Version'] == ver), ['Ensembl', 'Mayne']]
        tot = len(set(mm.Ensembl))
        overlap = len(set(get_deg_sig(tissue, fdr).ensemblID) & 
                      set(mm.Ensembl))
        xx = overlap / tot 
        ww.append(xx)
        print("There is %d (%.1f%%) overlap between %s and %s from Mayne!" % 
              (overlap, xx* 100, tissue_annotation(tissue), ver))
    print("")
    zz[tissue_annotation(tissue)] = ww

In [ ]:
dfm = pd.DataFrame.from_dict(zz)
dfm['Method'] = mayne.Version.unique()
dfm = dfm.set_index('Method')
dfm.to_csv('brainseq_overlap_mayne_fraction.txt', sep='\t')
dfm

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(6, 5))
plt.rcParams.update({'font.size': 18})
im, cbar = heatmap(dfm, dfm.index, dfm.columns, ax=ax,
                   cmap="YlGn", cbarlabel="Mayne et al. Comparison")
texts = annotate_heatmap(im, valfmt="{x:.2f}")

fig.tight_layout()
plt.savefig('brainseq_overlap_mayne_heatmap.png')
plt.savefig('brainseq_overlap_mayne_heatmap.pdf')
plt.savefig('brainseq_overlap_mayne_heatmap.svg')
plt.show()

In [ ]:
df = brainseq_shared_degs()

mm = mayne.loc[(mayne['Version'] == 'all'), ['Ensembl', 'Mayne']]
mm = get_biomart().merge(df, left_on='ensembl_gene_id', right_index=True)\
                  .merge(mm.set_index('Ensembl'), left_on="ensembl_gene_id", 
                         right_index=True, how='right')\
                  .set_index('external_gene_name')
mm.to_csv('brainseq_overlap_mayne_all_genes.csv')

In [ ]:
mm

## Gershoni and Pietrokovski (GTEx) Comparison

In [ ]:
gtex_file = here('input/public_results/extract_geneLists/_m/gp_gtex_sex_diff_genes.csv')
gtex = pd.read_csv(gtex_file).rename(columns={'Ensembl': 'Gencode'}).drop('Gene', axis=1)
gtex['Ensembl'] = gtex.Gencode.str.replace("\\..*", "", regex=True)
gtex.set_index('Ensembl', inplace=True)
gtex.shape

In [ ]:
zz = dict()
for tissue in ['caudate', 'dlpfc', 'hippocampus']:
    ww = []; yy = []
    fdr = 0.05
    for col in gtex.drop(['Symbol', 'Gencode'], axis=1).columns:
        gtex.loc[gtex[col] != 0, col] = 1
        tot = len(set(gtex[(gtex[col] != 0)].index))
        overlap = len(set(get_deg_sig(tissue, fdr).ensemblID) & 
                      set(gtex[(gtex[col] != 0)].index))
        xx = overlap / tot 
        ww.append(xx)
        print("There is %d (%.1f%%) overlap between %s and %s GTEx!" % 
              (overlap, xx* 100, tissue_annotation(tissue), col))
    print("")
    zz[tissue_annotation(tissue)] = ww

In [ ]:
dfg = pd.DataFrame.from_dict(zz)
dfg['Tissue'] = gtex.drop(['Symbol', 'Gencode'], axis=1).columns
dfg = dfg.set_index('Tissue')
dfg.to_csv('brainseq_overlap_gtex_fraction.txt', sep='\t')
dfg.head()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(36, 8))
plt.rcParams.update({'font.size': 12})
im, cbar = heatmap(dfg.T, dfg.columns, dfg.index, ax=ax,
                   cmap="YlGn", 
                   cbarlabel="Gershoni and Pietrokovski Comparison")
texts = annotate_heatmap(im, valfmt="{x:.2f}")

fig.tight_layout()
plt.savefig('brainseq_overlap_gtex_heatmap_full.png')
plt.savefig('brainseq_overlap_gtex_heatmap_full.pdf')
plt.savefig('brainseq_overlap_gtex_heatmap_full.svg')
plt.show()

In [ ]:
brain = dfg[(dfg.index.str.contains('Brain'))].copy()
fig, ax = plt.subplots(figsize=(18, 8))
plt.rcParams.update({'font.size': 18})
im, cbar = heatmap(brain.T, brain.columns, brain.index, ax=ax,
                   cmap="YlGn", 
                   cbarlabel="Gershoni and Pietrokovski Comparison")
texts = annotate_heatmap(im, valfmt="{x:.2f}")

fig.tight_layout()
plt.savefig('brainseq_overlap_gtex_heatmap.png')
plt.savefig('brainseq_overlap_gtex_heatmap.pdf')
plt.savefig('brainseq_overlap_gtex_heatmap.svg')
plt.show()

In [ ]:
df = brainseq_shared_degs()

gg = get_biomart().merge(df, left_on='ensembl_gene_id', right_index=True)\
             .merge(gtex.drop(['Symbol', 'Gencode'], axis=1), 
                    left_on='ensembl_gene_id', 
                    right_index=True, how='outer')\
             .set_index('external_gene_name')
gg.to_csv('brainseq_overlap_gtex_genes.csv')

## Session information

In [ ]:
session_info.show()